# Custom solvers
In this example, we show how to define custom solvers. Our system
will again be silicon, because we are not very imaginative

In [1]:
using DFTK, LinearAlgebra

a = 10.26
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]]

# We take very (very) crude parameters
model = model_LDA(lattice, atoms)
kgrid = [1, 1, 1]
Ecut = 5
basis = PlaneWaveBasis(model, Ecut; kgrid=kgrid);

We define our custom fix-point solver: simply a damped fixed-point

In [2]:
function my_fp_solver(f, x0, max_iter; tol)
    mixing_factor = .7
    x = x0
    fx = f(x)
    for n = 1:max_iter
        inc = fx - x
        if norm(inc) < tol
            break
        end
        x = x + mixing_factor * inc
        fx = f(x)
    end
    (fixpoint=x, converged=norm(fx-x) < tol)
end;

Our eigenvalue solver just forms the dense matrix and diagonalizes
it explicitly (this only works for very small systems)

In [3]:
function my_eig_solver(A, X0; maxiter, tol, kwargs...)
    n = size(X0, 2)
    A = Array(A)
    E = eigen(A)
    λ = E.values[1:n]
    X = E.vectors[:, 1:n]
    (λ=λ, X=X, residual_norms=[], iterations=0, converged=true, n_matvec=0)
end;

Finally we also define our custom mixing scheme. It will be a mixture
of simple mixing (for the first 2 steps) and than default to Kerker mixing.

In [4]:
struct MyMixing
    α  # Damping parameter
end
MyMixing() = MyMixing(0.7)

function DFTK.mix(mixing::MyMixing, basis, ρin::RealFourierArray, ρout::RealFourierArray; n_iter, kwargs...)
    if n_iter <= 2
        # Just do simple mixing
        ρin + mixing.α * (ρout - ρin)
    else
        # Use the KerkerMixing from DFTK
        DFTK.mix(KerkerMixing(α=mixing.α), basis, ρin, ρout; kwargs...)
    end
end

That's it! Now we just run the SCF with these solvers

In [5]:
scfres = self_consistent_field(basis;
                               tol=1e-8,
                               solver=my_fp_solver,
                               eigensolver=my_eig_solver,
                               mixing=MyMixing());

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   Diag
---   ---------------   ---------   --------   ----
  1   -7.234023541161         NaN   3.18e-01    0.0 
  2   -7.246507811113   -1.25e-02   1.44e-01    0.0 
  3   -7.248791516559   -2.28e-03   6.56e-02    0.0 
  4   -7.249075839954   -2.84e-04   3.61e-02    0.0 
  5   -7.249169882758   -9.40e-05   2.00e-02    0.0 
  6   -7.249200947283   -3.11e-05   1.12e-02    0.0 
  7   -7.249211349160   -1.04e-05   6.34e-03    0.0 
  8   -7.249214911154   -3.56e-06   3.64e-03    0.0 
  9   -7.249216164595   -1.25e-06   2.12e-03    0.0 
 10   -7.249216618587   -4.54e-07   1.25e-03    0.0 
 11   -7.249216787686   -1.69e-07   7.50e-04    0.0 
 12   -7.249216852288   -6.46e-08   4.57e-04    0.0 
 13   -7.249216877514   -2.52e-08   2.81e-04    0.0 
 14   -7.249216887544   -1.00e-08   1.75e-04    0.0 
 15   -7.249216891591   -4.05e-09   1.10e-04    0.0 


Note that the default convergence criterion is on the difference of
energy from one step to the other; when this gets below `tol`, the
"driver" `self_consistent_field` artificially makes the fixpoint
solver think it's converged by forcing `f(x) = x`. You can customize
this with the `is_converged` keyword argument to
`self_consistent_field`.